<a href="https://colab.research.google.com/github/riverdogcabin/PSDS4900/blob/main/wunderground_retrieve_missing_observations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
from pprint import pprint
import pandas as pd
from datetime import datetime
from time import sleep

In [ ]:
# Set up API call URL
# weatherreadings = {}
url = 'https://api.weather.com/v2/pws/history/hourly?stationId={stationid}&format=json&units={units}&date={date}&apiKey={apikey}&numericPrecision={numeric_precision}'
# url = 'https://api.weather.com/v2/pws/observations/hourly/7day?stationId={stationid}&format=json&units={units}&apiKey={apikey}&numericPrecision={numeric_precision}'
# url = "https://api.weather.com/v2/pws/observations/current?stationId=REPLACESTATIONID&format=json&units=REPLACEUNITS&numericPrecision=decimal&apiKey=REPLACEAPI"


In [ ]:
# Get Config Data from config.json
with open('/content/drive/MyDrive/Capstone/PSDS4900/config.json') as configuration:
  config = json.load(configuration).get('WU') #config for WeatherUnderground
  apikey = config["apikey"]
  my_stationid = config["stationid"]
  units = config["units"]
  numeric_precision = config["numeric_precision"]

In [ ]:
url = url.format(stationid = '{stationid}',date='{date}',units=units,apikey=apikey,numeric_precision=numeric_precision)
# print(url)
# station_file = '/content/drive/MyDrive/Capstone/PSDS4900/wu_stations.txt'
observation_file = '/content/drive/MyDrive/Capstone/PSDS4900/wu_data/wu_observations.csv'
missing_dates = []
drop_columns = ['lat','lon','units','tz','obsTimeUtc','obsTimeLocal']

In [ ]:
# JSON comes back with measurements nested inside 'metrics', need to flatten
def flatten_wu_result(wu_result,units='metric'):
  try:
    observation = wu_result.pop(units)
    wu_result.update(observation)
    wu_result.update({'units':units})
  except:
    return wu_result

In [ ]:
# function to get the JSON data from Weather Underground
def get_wu_data_specific_date(stationid,date,url):
  url = url.format(stationid=stationid,date=date)
  print("Retrieving data for {} and date {}".format(stationid,date),end=': ')
  response = requests.get(url) # need to add error handling for invalid data

  if response.status_code == 200: # If the response code is 200 (sucessful)
      print("Success")
      weatherdata = response.json()
      # pprint(weatherdata)
      for observation in weatherdata.get('observations'):
        flatten_wu_result(observation)
      return weatherdata.get('observations')
  elif response.status_code == 204: # 204 returns a blank response so inform the user
      print("API Error 204: No Data Found for specific query.")
      exit()

  elif response.status_code >200 & response.status_code <1000: # Handles all other error code numbers
      # result = response.json()
      try:
          print("API Error " + str(response.status_code) + ": " + response.text)
      except: # sometimesthe error messae is in a different nest
          print("API Error " + str(response.status_code) + ": " + result["errors"][0]["message"])
      exit()

  else: # Catch any other errors
      print("Unspecified error")
      print(response.text)

In [ ]:
def get_stations(stationfile):
  station_list = []
  with open(stationfile) as infile:
    for l in infile.readlines():
      if(l.find('#') == -1):
        station_list.append(l.strip())
  return station_list

In [ ]:
# def get_mission_station_dates(observation_file):
df = pd.read_csv(observation_file)
df['date'] = pd.to_datetime(df['epoch'],unit='s').dt.date
# df['date'] = df['date'].dt.date()


In [ ]:
all_stations = set(df.stationID.unique())
# all_stations

In [ ]:
grouped = df[['date','stationID']].groupby('date')
result_dict = {}
for group in grouped.groups:
  result_dict[group.strftime('%Y%m%d')] = all_stations - set(grouped.get_group(group).stationID.unique())

In [ ]:
result_dict

{'20210316': {'KCOGLENW12',
  'KCOGLENW6',
  'KCOMONUM250',
  'KCOSEDAL14',
  'KCOSTEAM93'},
 '20210317': {'KCOGLENW12', 'KCOGLENW6', 'KCOMONUM250', 'KCOSEDAL14'},
 '20210318': {'KCOGLENW12', 'KCOGLENW6', 'KCOMONUM250', 'KCOSEDAL14'},
 '20210319': {'KCOGLENW12',
  'KCOGLENW6',
  'KCOMONUM250',
  'KCOSEDAL14',
  'KCOSTEAM93'},
 '20210320': {'KCOGLENW12',
  'KCOGLENW6',
  'KCOGRAND254',
  'KCOMONUM250',
  'KCOSEDAL14',
  'KCOSTEAM93'},
 '20210321': {'KCOGLENW12',
  'KCOGLENW6',
  'KCOGRAND254',
  'KCOMONUM250',
  'KCOSEDAL14',
  'KCOSTEAM93'},
 '20210322': {'KCOFORTC111',
  'KCOFORTC165',
  'KCOFORTC379',
  'KCOFORTC421',
  'KCOFORTC425',
  'KCOGENOA4',
  'KCOGLENW12',
  'KCOGRAND197',
  'KCOGRAND229',
  'KCOGRAND248',
  'KCOGRAND254',
  'KCOGRAND304',
  'KCOMONUM250',
  'KCOSTEAM93'},
 '20210323': {'KCOFORTC111',
  'KCOFORTC165',
  'KCOFORTC379',
  'KCOFORTC421',
  'KCOFORTC425',
  'KCOGENOA4',
  'KCOGLENW12',
  'KCOGRAND197',
  'KCOGRAND229',
  'KCOGRAND248',
  'KCOGRAND254',
  'KCOGRA

In [ ]:
observations_df = pd.DataFrame()
for date in result_dict.keys():
  for station in result_dict.get(date):
    observations_df = observations_df.append(pd.DataFrame(get_wu_data_specific_date(station,date,url)))
    sleep(1)
  sleep(10)

Retrieving data for KCOFORTC379 and date 20210316: Success
Retrieving data for KCOFORTC111 and date 20210316: Success
Retrieving data for KCOMONUM250 and date 20210316: Success
Retrieving data for KCOGLENW6 and date 20210316: Success
Retrieving data for KCOMONUM140 and date 20210316: Success
Retrieving data for KCOFORTC425 and date 20210316: Success
Retrieving data for KCOGENOA4 and date 20210316: Success
Retrieving data for KCOGRAND254 and date 20210316: Success
Retrieving data for KCOSEDAL14 and date 20210316: Success
Retrieving data for KCOGLENW12 and date 20210316: Success
Retrieving data for KCOSTEAM93 and date 20210316: Success
Retrieving data for KCOGRAND229 and date 20210316: Success
Retrieving data for KCOFORTC421 and date 20210316: Success
Retrieving data for KCOGRAND304 and date 20210316: Success
Retrieving data for KCOGRAND197 and date 20210316: Success
Retrieving data for KCOGRAND248 and date 20210316: Success
Retrieving data for KCOFORTC165 and date 20210316: Success
Retr

In [ ]:
if(observations_df.shape == (0,0)):
  print("Observations unavailable for these stations on these dates")
else:
  observations_df.drop(columns=drop_columns,inplace=True)
  observations_df.to_csv('/content/drive/MyDrive/Capstone/PSDS4900/wu_data/{}_missing_observations.csv'.format(datetime.now().date().strftime('%Y%m%d')),index=False)
  print('/content/drive/MyDrive/Capstone/PSDS4900/wu_data/{}_missing_observations.csv written\n'.format(datetime.now().date().strftime('%Y%m%d')))  

/content/drive/MyDrive/Capstone/PSDS4900/wu_data/20210411_missing_observations.csv written

